In [1]:
import pandas as pd
import os
import sys
import altair as alt
data_path = os.path.abspath('../outbreak_data')
sys.path.append(data_path)
import outbreak_common

In [2]:
states = pd.read_csv('camping_states.csv') 

In [3]:
# rolling_average, weekly // adding a column to the df which contains the running mean of numIncrease
num_days = 7
# groupby // since we have multiple states, we need to group by locality so the running_average is 
# calculated individiually

rolling_states = []

for state, data, in states.groupby('admin1'):
    col_name = 'confirmed_numIncrease_%rdayRolling'%num_days
    
    data[col_name] = outbreak_common.rolling_average(data.confirmed_numIncrease, num_days)
    rolling_states.append(data)

In [4]:
rolling_states = pd.concat(rolling_states)

In [5]:
# pd.to_datetime // applying to_datetime will allow numpy to sort the dates for visualization
rolling_states['date'] = rolling_states['date'].apply(pd.to_datetime)

In [6]:
# reset_index // resetting the index since groupby split and reindexed by state
rolling_states.reset_index(drop=True, inplace=True)

In [7]:
# to_json // writing the data in json format for quick visualization
url = 'rolling_viz.json'
rolling_states.to_json(url, orient='records')

In [8]:
# 7-day rolled (avg) feature viz // avg new amount of covid cases, per 7 days
rolling = alt.Chart(url).mark_line().encode(
    x='date:T',
    y='confirmed_numIncrease_7dayRolling:Q',
    color='admin1:N'
).interactive()

In [9]:
# base feature viz // amount of new covid cases
base = alt.Chart(url).mark_line().encode(
    x='date:T',
    y='confirmed_numIncrease:Q',
    color='admin1:N'
).interactive()

### Understanding popular space/time loci of 'Camping/Outdoors/Recreational' trips
Colorado
 - https://oedit.colorado.gov/sites/coedit/files/2021-08/2020_longwoods_travel_usa_colorado_report_1.pdf
 - Popular seasons: Sep-Dec(29%), July-Sep(28%), Jan-Mar(25%) Apr-Jun(18%) 

In [10]:
# rules // added dates of popular tourism dates
rules = alt.Chart(pd.DataFrame({
  'date': ['2022-9-01', '2021-09-01', '2020-09-01'],
  'color': ['black', 'black', 'black']
})).mark_rule().encode(
  x='date:T',
  color=alt.Color('color:N', scale=None)
)

#### Plot of numIncrease vs  numIncrease_7dayRolling (confirmed cases)

In [11]:
(rules + base) | (rules + rolling)

alt.HConcatChart(...)